In [1]:
# in reference to:
# https://towardsdatascience.com/scraping-reddit-with-praw-76efc1d1e1d9
# https://praw.readthedocs.io/en/latest/code_overview/praw_models.html
# https://www.reddit.com/wiki/search

In [1]:
from os.path import isfile
import praw
import pandas as pd
from time import sleep

# Get credentials from DEFAULT instance in praw.ini
# Use your own tokens
reddit = praw.Reddit(
    client_id="VSZN7S6ns1a21Q",
    client_secret="bjsqugp82k3cBGTh2ZFH72-hMxE",
    user_agent="Scraper",
    username="Scraper1",
    password="Scraper1"
)


class SubredditScraper:

    def __init__(self, query, sort='new', lim=900, mode='w'):
        self.query = query
        self.sort = sort
        self.lim = lim
        self.mode = mode

        print(
            f'SubredditScraper instance created with values '
            f'query = {query}, sort = {sort}, lim = {lim}, mode = {mode}')

    def set_sort(self):
        if self.sort == 'new':
            return self.sort, reddit.subreddit('all').search(self.query, sort='new', time_filter='month')
        elif self.sort == 'top':
            return self.sort, reddit.subreddit('all').search(self.query, sort='top', time_filter='month')
        elif self.sort == 'hot':
            return self.sort, reddit.subreddit('all').search(self.query, sort='hot', time_filter='month')
        else:
            self.sort = 'relevance'
            print('Sort method was not recognized, defaulting to relevance.')
            return self.sort, reddit.subreddit('all').search(self.query, time_filter='month')

    def get_posts(self, printer=False):

        sub_dict = {
            'game_title': [], 'selftext': [], 'title': [], 'id': [], 'sorted_by': [],
            'num_comments': [], 'score': [], 'ups': [], 'downs': []}
        csv = f'game_titles_2.csv'

        # Attempt to specify a sorting method.
        sort, subreddit = self.set_sort()

        # Set csv_loaded to True if csv exists since you can't evaluate the
        # truth value of a DataFrame.
        df, csv_loaded = (pd.read_csv(csv), 1) if isfile(csv) else ('', 0)

        print(f'csv = {csv}')
        print(f'After set_sort(), sort = {sort} and query = {self.query}')
        print(f'csv_loaded = {csv_loaded}')

        print(f'Collecting information from r/all for search term "{self.query}".')
        for post in subreddit:
            # Save any unique, non-stickied posts with descriptions to sub_dict.
            sub_dict['game_title'].append(self.query)
            sub_dict['selftext'].append(post.selftext)
            sub_dict['title'].append(post.title)
            sub_dict['id'].append(post.id)
            sub_dict['sorted_by'].append(sort)
            sub_dict['num_comments'].append(post.num_comments)
            sub_dict['score'].append(post.score)
            sub_dict['ups'].append(post.ups)
            sub_dict['downs'].append(post.downs)
            sleep(0.1)

        sub_dict_final = {
            'game_title': [self.query],  
            'score': [sum(sub_dict['score'])], 
            'ups': [sum(sub_dict['ups'])], 
            'downs': [sum(sub_dict['downs'])]}
        
        # print(sub_dict)
        new_df = pd.DataFrame(sub_dict_final)
        
        # decide whether or no to print
        if printer is True:
            print(
                f'{len(sub_dict)} posts were collected but they were not '
                f'added to {csv} because mode was set to "{self.mode}"')
            return

        # Add new_df to df if df exists then save it to a csv.
        if 'DataFrame' in str(type(df)) and self.mode == 'w':
            pd.concat([df, new_df], axis=0, sort=0).to_csv(csv, index=False)
            print(
                f'{len(new_df)} new posts collected and added to {csv}')
        elif self.mode == 'w':
            new_df.to_csv(csv, index=False)
            print(f'{len(new_df)} posts collected and saved to {csv}')
        else:
            print(
                f'{len(new_df)} posts were collected but they were not '
                f'added to {csv} because mode was set to "{self.mode}"')


#if __name__ == '__main__':
    #SubredditScraper('statistics', lim=997, mode='w', sort='new').get_posts()
    #SubredditScraper('Over Watch', lim=997, sort='new').get_posts()

In [2]:
all_titles = pd.read_csv("All_Games.csv", error_bad_lines=False)

b'Skipping line 252: expected 8 fields, saw 9\nSkipping line 284: expected 8 fields, saw 10\nSkipping line 396: expected 8 fields, saw 9\nSkipping line 609: expected 8 fields, saw 9\nSkipping line 610: expected 8 fields, saw 9\nSkipping line 628: expected 8 fields, saw 9\nSkipping line 696: expected 8 fields, saw 9\nSkipping line 1561: expected 8 fields, saw 9\nSkipping line 2536: expected 8 fields, saw 9\nSkipping line 2650: expected 8 fields, saw 9\nSkipping line 2726: expected 8 fields, saw 9\nSkipping line 2773: expected 8 fields, saw 9\nSkipping line 2891: expected 8 fields, saw 11\nSkipping line 2892: expected 8 fields, saw 10\nSkipping line 2983: expected 8 fields, saw 9\nSkipping line 3005: expected 8 fields, saw 9\nSkipping line 3094: expected 8 fields, saw 10\nSkipping line 3119: expected 8 fields, saw 9\nSkipping line 3141: expected 8 fields, saw 10\nSkipping line 3151: expected 8 fields, saw 13\nSkipping line 3171: expected 8 fields, saw 17\nSkipping line 3289: expected 8 f

In [3]:
# [[0, 1283], [1283, 2566], [2566, 3849], [3849, 5132], [5132, 6417]]

for i in range(2566, 3849):
    print('Scraping: (' + str(i) + ')')
    title = all_titles.title[i]
    SubredditScraper(str(title), lim=997, sort='new').get_posts()

Scraping: (2566)
SubredditScraper instance created with values query = Skate 2, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Skate 2
csv_loaded = 0
1 posts collected and saved to game_titles_2.csv
Scraping: (2567)
SubredditScraper instance created with values query = Skate 3, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Skate 3
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2568)
SubredditScraper instance created with values query = Ski-Doo: Snowmobile Challenge, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Ski-Doo: Snowmobile Challenge
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2569)
SubredditScraper instance created with values query = Skylanders: Spyro's Adventure, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Sk

1 new posts collected and added to game_titles_2.csv
Scraping: (2589)
SubredditScraper instance created with values query = Soulcalibur IV, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Soulcalibur IV
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2590)
SubredditScraper instance created with values query = Soulcalibur V, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Soulcalibur V
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2591)
SubredditScraper instance created with values query = Spec Ops: The Line, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Spec Ops: The Line
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2592)
SubredditScraper instance created with values query = Spider-Man 3, sort = new, lim = 997, mode = w
csv = game_titles_2.csv

1 new posts collected and added to game_titles_2.csv
Scraping: (2612)
SubredditScraper instance created with values query = Stormrise, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Stormrise
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2613)
SubredditScraper instance created with values query = Street Fighter IV, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Street Fighter IV
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2614)
SubredditScraper instance created with values query = Street Fighter X Tekken, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Street Fighter X Tekken
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2615)
SubredditScraper instance created with values query = Street Rod, sort = new, lim = 997, mode = w
csv = game_titles

1 new posts collected and added to game_titles_2.csv
Scraping: (2634)
SubredditScraper instance created with values query = Supremacy MMA, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Supremacy MMA
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2635)
SubredditScraper instance created with values query = Surf's Up, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Surf's Up
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2636)
SubredditScraper instance created with values query = Suzumiya Haruhi no Tsuisou, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Suzumiya Haruhi no Tsuisou
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2637)
SubredditScraper instance created with values query = Syberia, sort = new, lim = 997, mode = w
csv = game_titles_2.cs

1 new posts collected and added to game_titles_2.csv
Scraping: (2658)
SubredditScraper instance created with values query = The Chronicles of Narnia: Prince Caspian, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = The Chronicles of Narnia: Prince Caspian
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2659)
SubredditScraper instance created with values query = The Chronicles of Riddick: Assault on Dark Athena, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = The Chronicles of Riddick: Assault on Dark Athena
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2660)
SubredditScraper instance created with values query = The Club, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = The Club
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (2661)
SubredditScraper in

...

1 new posts collected and added to game_titles_2.csv
Scraping: (3753)
SubredditScraper instance created with values query = Human Resource Machine, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Human Resource Machine
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3754)
SubredditScraper instance created with values query = Huntdown, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Huntdown
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3755)
SubredditScraper instance created with values query = Hyper Sentinel, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Hyper Sentinel
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3756)
SubredditScraper instance created with values query = Hyper Sports R, sort = new, lim = 997, mode = w
csv = game_titles_2.csv

1 new posts collected and added to game_titles_2.csv
Scraping: (3778)
SubredditScraper instance created with values query = It's Spring Again, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = It's Spring Again
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3779)
SubredditScraper instance created with values query = Ittle Dew 2+, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Ittle Dew 2+
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3780)
SubredditScraper instance created with values query = Jang-Navi Mahjong Online, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Jang-Navi Mahjong Online
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3781)
SubredditScraper instance created with values query = J.B. Harold Murder Club, sort = new, lim = 997, mode 

1 new posts collected and added to game_titles_2.csv
Scraping: (3801)
SubredditScraper instance created with values query = The King's Bird, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = The King's Bird
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3802)
SubredditScraper instance created with values query = Kirby Star Allies, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Kirby Star Allies
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3803)
SubredditScraper instance created with values query = Kitten Squad, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Kitten Squad
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3804)
SubredditScraper instance created with values query = Knight Terrors, sort = new, lim = 997, mode = w
csv = game_titles_2.csv

1 new posts collected and added to game_titles_2.csv
Scraping: (3824)
SubredditScraper instance created with values query = Lichtspeer: Double Speer Edition, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Lichtspeer: Double Speer Edition
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3825)
SubredditScraper instance created with values query = Light Fall, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Light Fall
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3826)
SubredditScraper instance created with values query = Light Fingers, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Light Fingers
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
Scraping: (3827)
SubredditScraper instance created with values query = Lightseekers, sort = new, lim = 997, mode = w
csv

1 new posts collected and added to game_titles_2.csv
Scraping: (3848)
SubredditScraper instance created with values query = Maid of Sker, sort = new, lim = 997, mode = w
csv = game_titles_2.csv
After set_sort(), sort = new and query = Maid of Sker
csv_loaded = 1
1 new posts collected and added to game_titles_2.csv
